In [ ]:
import tensorflow as tf
from ParticleNet import get_particle_net
# Logging and learning control
import logging
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard


gpu = True
if not gpu:
# Establecer para que TensorFlow solo use CPU
  tf.config.set_visible_devices([], 'GPU')

# Verificar si solo se está usando CPU
  print("Dispositivos físicos disponibles:", tf.config.list_physical_devices('GPU'))

# Tu código de TensorFlow aquí
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH

# !export LD_LIBRARY_PATH=/mnt/c/Users/olmic/Desktop/Personal/Máster/TFM/Código/TFM-CTLearn/cudnn/cudnn-linux-x86_64-8.9.2.26_cuda11-archive/lib:$LD_LIBRARY_PATH
# !export CUDNN_INCLUDE_DIR=/mnt/c/Users/olmic/Desktop/Personal/Máster/TFM/Código/TFM-CTLearn/cudnn/cudnn-linux-x86_64-8.9.2.26_cuda11-archive/include

In [ ]:
import os

def generate_call_backs(log_dir, extra_metric = None, reduce_lr_options = dict({"reduce_lr":True}), early_stopping_options = dict({"early_stop":True}), checkpoint_options = dict({"checkpoint":True})):
    # Configurar el logger
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    log_file = log_dir+'training.log'
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.INFO)

    # Crear un formato para los logs
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)

    # Añadir el manejador de archivo al logger
    logger.addHandler(file_handler)

    # Callback personalizado para logging
    class LoggingCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            if extra_metric is not None:
                logger.info(f"Epoch {epoch + 1}, Loss: {logs['loss']}, Accuracy: {logs['accuracy']}, {extra_metric}: {logs[extra_metric]}, Val Loss: {logs['val_loss']}, Val Accuracy: {logs['val_accuracy']}, Val {extra_metric}: {logs['val_'+extra_metric]}")
            else:
                logger.info(f"Epoch {epoch + 1}, Loss: {logs['loss']}, Accuracy: {logs['accuracy']}, Val Loss: {logs['val_loss']}, Val Accuracy: {logs['val_accuracy']}")

    # Configurar TensorFlow para usar el logger de Python
    tf.get_logger().setLevel('INFO')

    if reduce_lr_options.get("reduce_lr", False):
        # Callback para reducir el learning rate
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor = reduce_lr_options.get("monitor", "val_loss"),
            factor = reduce_lr_options.get("factor", 0.1),
            patience = reduce_lr_options.get("patience", 5),
            verbose = 1,
            mode = reduce_lr_options.get("mode", "auto"),
            min_delta = reduce_lr_options.get("min_delta", 0.0001),
            cooldown = reduce_lr_options.get("cooldown",2),
            min_lr = reduce_lr_options.get("min_lr",0.00001)
        )
    
    if early_stopping_options.get("early_stop", False):
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor = early_stopping_options.get("monitor", "val_loss"),
            patience = early_stopping_options.get("patience", 5),
            restore_best_weights = early_stopping_options.get("restore_best_weights", True)
        )

    # Configurar el callback de TensorBoard
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Prepare model model saving directory.
    save_dir = log_dir + 'model_checkpoints'
    model_name = '%s_model.{epoch:03d}.h5' % 'PCT_low_data'
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    filepath = os.path.join(save_dir, model_name)

    if checkpoint_options.get("checkpoint", False):
        checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                    monitor=checkpoint_options.get("monitor", 'val_loss'),
                                    verbose=1,
                                    save_best_only=True,
                                    mode=checkpoint_options.get("mode", 'auto'))

    return [LoggingCallback(), reduce_lr, early_stopping, tensorboard_callback, checkpoint]


In [ ]:

# Generación de datos
from DataLoader import PCDataGenerator
import h5py
import numpy as np

data_path = "../data/dataset_500_rel_coords.h5"
# Dividir el dataset en train, test y validation
with h5py.File(data_path, 'r') as f:
    all_indices = list(f.keys())

# Suponiendo que quieres una división 70% train, 15% validation, 15% test
np.random.seed(1234)
np.random.shuffle(all_indices)
train_split = int(0.7 * len(all_indices))
val_split = int(0.85 * len(all_indices))

train_indices = all_indices[:train_split]
val_indices = all_indices[train_split:val_split]
test_indices = all_indices[val_split:]

# Crear generadores para cada partición
batch_size = 16
train_data = PCDataGenerator(data_path, train_indices, batch_size=batch_size, shuffle=True)
val_data = PCDataGenerator(data_path, val_indices, batch_size=batch_size, shuffle=True)
test_data = PCDataGenerator(data_path, test_indices, batch_size=batch_size, shuffle=False)

In [ ]:
import yaml

with open('PN_config.yml', 'r') as myconfig:
  config = yaml.load(myconfig, Loader=yaml.Loader)
  
model_config = config["Model Parameters"]
model_config["model_directory"] = "./"
model, inputs = get_particle_net(train_data, model_config)
model.summary()

In [ ]:
losses = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
AUC = tf.keras.metrics.AUC()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
    tf.keras.metrics.AUC(name="auc"),
]
model.compile(loss=losses,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=metrics)

In [ ]:
log_dir = '../logs/PC_low_data_test_weighted_auc/'
callbacks_weighted = generate_call_backs(log_dir, extra_metric="auc")
# Entrenar el modelo
history = model.fit(
    train_data,
    validation_data=val_data,
    class_weight=train_data.class_weight,
    epochs=10,
    callbacks=callbacks_weighted
)

## Evaluación

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score

# Asume que tienes un modelo entrenado llamado `model`
# y un generador de datos de prueba llamado `test_generator`

# Realiza predicciones sobre el conjunto de datos de prueba
y_pred_probs = model.predict(test_data)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Obtener las etiquetas reales
y_true_vectors = np.concatenate([y for x, y in test_data], axis=0)
y_true_classes = np.argmax(y_true_vectors, axis=1)

76/76 [==============================] - 6s 67ms/step


In [ ]:

# Calcula la matriz de confusión
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Calcula precision, recall, f1-score, y otras métricas
classification_rep = classification_report(y_true_classes, y_pred_classes)
print('Classification Report:')
print(classification_rep)

Confusion Matrix:
[[ 14 997]
 [  1 204]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.01      0.03      1011
           1       0.17      1.00      0.29       205

    accuracy                           0.18      1216
   macro avg       0.55      0.50      0.16      1216
weighted avg       0.80      0.18      0.07      1216



In [15]:
import tensorflow as tf
from ParticleNet import MaskCoordShiftLayer, AddShifttoCoordLayer, BatchDistanceMatrix, BatchNormalLayerfts, Get_Knn_Index, knn, Get_Knn_fts, SqueezeLayer, ReduceLayer
from tensorflow.keras.utils import custom_object_scope



with custom_object_scope({'MaskCoordShiftLayer': MaskCoordShiftLayer,
                          "AddShifttoCoordLayer": AddShifttoCoordLayer,
                          "BatchDistanceMatrix": BatchDistanceMatrix,
                          "BatchNormalLayerfts": BatchNormalLayerfts,
                          "Get_Knn_Index": Get_Knn_Index,
                          "knn": knn,
                          "Get_Knn_fts": Get_Knn_fts,
                          "SqueezeLayer": SqueezeLayer,
                          "ReduceLayer": ReduceLayer}):
    model = tf.keras.models.load_model('../logs/PC_low_data_test_weighted_auc/model_checkpoints/PCT_low_data_model.002.h5')

# Cargamos el modelo  logs/trial_complete_data/ctlearn_model
# y el modelo logs/PC_low_data_test/model_checkpoints/PCT_low_data_model.002.h5

# model = tf.keras.models.load_model('../logs/PC_low_data_test_weighted_auc/model_checkpoints/PCT_low_data_model.002.h5')
# model_2 = tf.keras.models.load_model('../logs/trial_complete_data/ctlearn_model')

# Mostramos el número de parámetros de cada modelo
print("Número de parámetros del modelo 1:", model.count_params())

# print("Número de parámetros del modelo 2:", model_2.count_params())

Número de parámetros del modelo 1: 409546


In [16]:
model.summary()

Model: "ParticleNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 mask (InputLayer)           [(None, 500, 1)]             0         []                            
                                                                                                  
 MaskCoordShiftLayer (MaskC  (None, 500, 1)               0         ['mask[0][0]']                
 oordShiftLayer)                                                                                  
                                                                                                  
 points (InputLayer)         [(None, 500, 2)]             0         []                            
                                                                                                  
 AddShifttoCoordLayer_layer  (None, 500, 2)               0         ['MaskCoordShiftLaye